In [ ]:
###
# Lab 7: String Sorting and Dynamic Programming
###

# Covers lectures 9a-b

'''
AGENDA
(1) Skribbl.io
(2) Radix sort (LSD vs MSD, stable vs unstable)
(3a) 1D DP
Examples: Fibonacci, change problem
(3b) 2D DP
Examples: Knapsack, string alignment
'''

In [ ]:
import numpy as np
from collections import defaultdict

# counting sort

a = [5, 3, 1, 3, 9]
R = 10 # for single digit sorting
N = len(a)
aux = np.zeros(N)
count = np.zeros(R + 1) # +1 for offset

def print_progress(count, aux):
    print('Count and Aux:')
    print('Count ( keys  ): ' + str(np.array([0] + [float(x) for x in range(len(count)-1)])))
    print('Count (entries): ' + str(count))
    print('Aux: ' + str(aux))

# iterate over a to populate count at the right places with the number of entries i.e. obtain frequency 
print('Original: ' + str(a))
for i in range(N):
    count[a[i]+1] += 1 

print_progress(count, aux)

# obtain cumulative distribution
for r in range(R): 
    count[r+1] += count[r] 

print_progress(count, aux)

# populate aux with information from count 
for i in range(N): 
    # count[a[i]] keeps track of where the next a[i] should go in aux
    aux[int(count[a[i]])] = a[i]
    count[a[i]] += 1 

print_progress(count, aux)
    
# copy
for i in range(N):
    a[i] = aux[i]

print('Updated original: ' + str(a))

# TODO: What is the runtime of this algorithm? Worst and best cases?

# O(n) or O(n+r)
# n for the auxillary array and r for the counting array

# TODO: What if you had the largest element as n^2? e.g. len(a) = 10, max(a) = 100

# Can make things more complicated because it has to go the length of all values so even if small array, 
# it's still going to 100

In [ ]:
def MSD_radix_string_sort(L, i):
    # base case (list must already be sorted), also accounts for '' empty string len = 0 case
    if len(L) <= 1:
        return L

    # divide into buckets based on length
    # akin to aux array from the previous problem
    done_bucket = []
    
    # akin to count array from the previous problem
    buckets = [ [] for x in range(27) ] # one for each letter in a-z

    # for each string in currently analyzed list
    # each string will be processed once 
    # TODO: how many times will this loop run as a function of original length L over all recursive calls?
    for s in L:
        # i denotes position in string (left to right)
        if i >= len(s):
            done_bucket.append(s)
        else:
            # ord converts a string to a numerical heirarchy that is comparable 
            # puts the string s in the sub-bucket index corresponding to that letter
            buckets[ord(s[i]) - ord('a')].append(s)
    
    # conquer (recursively sort buckets)
    # i = 0 --> i = R
    # R = alphabet, i needs to go the length of the longest word
    buckets = [MSD_radix_string_sort(b, i + 1) for b in buckets]

    # chain all buckets together
    # TODO: how many times will this loop run as a function of R = 27 buckets over all recursive calls?    
    current_bucket = []
    for blist in buckets:
        for b in blist:
            current_bucket.append(b)
            
    return done_bucket + current_bucket

L = ['joe', '', 'bob', 'rot', 'rocket', 'rock', 'z', 'mary', 'suzy', 'sue']
print(L)
L = MSD_radix_string_sort(L, 0)
print(L)

# R (length of longest word); i from 0 to length of longest word, say R
# L strings total
# b (base/radix, 27 for alphabet)
# TODO: What is the overall runtime of this sorting algorithm?
# O(R(b+L))


In [3]:
'''
1D DP:
We've seen: Fibonacci, BestChange, ... 

Problem: given n, find the number of different ways to write n as the sum of 1, 3, 4

Example: n = 5
5 = 1 + 1 + 1 + 1 + 1
  = 1 + 1 + 3
  = 1 + 3 + 1
  = 3 + 1 + 1
  = 1 + 4
  = 4 + 1

Step 1: Define recurrence

Step 2: Define base cases

Step 3: Iterate

Step 4: (optional) backtrace

'''
# (1) Recurrence and subproblems
# T_n = T_(n-1) + T_(n-3) + T_(n-4)

# T_5 = 6

# (2) Base case(s)
# how do you know how many you need?
# T_0 = 0
# T_1 = 1 // 1
# T_2 = 1 // 1 + 1
# T_3 = 2 // 1 + 1 + 1, 3

import numpy as np
def special_sum(n):
    # initialize working array (akin to aux, but we're not sorting)
    T = np.zeros(n+1)
    
    # TODO: define base cases
    T[0] = 1 # If you put 0 it ends up being fibonnacci sequence
    T[1] = 1
    T[2] = 1
    T[3] = 2
    
    # TODO: define recurrence
    for i in range(4, n+1):
        T[i] = T[i-1] + T[i-3] + T[i-4]    
    return T[n]

for i in range(4, 10):
    print('The number of ways to sum to ' + str(i) + \
          ' using 1, 3, and 4 (noncommutatively) is ' + str(int(special_sum(i))) + '.')

# TODO: What is the runtime of this algorithm as a function of n?
# O(n)
# Memory?
# O(n)

# TODO: What about backtrace? 

The number of ways to sum to 4 using 1, 3, and 4 (noncommutatively) is 4.
The number of ways to sum to 5 using 1, 3, and 4 (noncommutatively) is 6.
The number of ways to sum to 6 using 1, 3, and 4 (noncommutatively) is 9.
The number of ways to sum to 7 using 1, 3, and 4 (noncommutatively) is 15.
The number of ways to sum to 8 using 1, 3, and 4 (noncommutatively) is 25.
The number of ways to sum to 9 using 1, 3, and 4 (noncommutatively) is 40.


In [ ]:
'''
Brute Force LCS
using APPLETOMATOORANGE and ANT as the two strings being compared

n APPLETOMATOORANGE
m ANT

n APPLEORANGETOMATO
m A NT

n APPLEORANGETOMATO
m AN T 

... and some extreme cases

n APPLEORANGETOMATO
m                 ANT 

n   APPLEORANGETOMATO
m ANT                 

leads to
(n+m choose m) = (n + m)!/(n!m!) (yikes!)

'''

In [4]:
'''
Rosalind
Longest Common Subsequence Problem
Given: Two strings.

Return: A longest common subsequence of these strings.

Sample Dataset
AACCTTGG
ACACTGTGA

Sample Output
AACTGG
'''

# LCS and backtrace
with open('/Users/KevinBu/Desktop/BMI2005_Algorithms_2020/Labs/rosalind/rosalind_ba5c.txt', 'r') as f:
    string1 = 'e' + f.readline().strip()
    string2 = 'e' + f.readline().strip()

def LCS(string1, string2):
    m = len(string1)-1
    n = len(string2)-1
    T = np.zeros(shape=[m+1,n+1])

    # base case
    for i in range(1,m+1):
        T[i,0] = 0

    for j in range(1,n+1):
         T[0,j] = 0
    
    for i in range(1,m+1):
        for j in range(1,n+1):
            if string1[i] == string2[j]:
                T[i,j] = T[i-1,j-1] + 1
            else:
                T[i,j] = max(T[i,j-1],T[i-1,j])
               
    return T[m,n]
        
LCS(string1,string2)

# Runtime is O(mn)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/KevinBu/Desktop/BMI2005_Algorithms_2020/Labs/rosalind/rosalind_ba5c.txt'

In [ ]:
# What about backtrace?

In [ ]:
def LCS_with_backtrace(string1, string2):
    string1 = 'e' + string1
    string2 = 'e' + string2
    m = len(string1)-1
    n = len(string2)-1
    T = np.zeros(shape=[m+1,n+1])
    b = np.zeros(shape=[m+1,n+1])
    # base case; 0 = no gain for using information from just one string
    for i in range(1,m+1):
        T[i][0] = 0
    for j in range(1,n+1):
        T[0][j] = 0

    for i in range(1,m+1):
        for j in range(1,n+1):
            if string1[i] == string2[j]:
                # T[i][j] = max(T[i-1][j-1]+1,T[i-1][j],T[i][j-1])
                if T[i-1][j-1] + 1 == max(T[i-1][j-1]+1,T[i-1][j],T[i][j-1]):
                    T[i][j] = T[i-1][j-1] + 1
                    b[i][j] = 1
            else:
                # T[i][j] = max(T[i-1][j],T[i][j-1])
                if T[i-1][j] == max(T[i-1][j],T[i][j-1]):
                    T[i][j] = T[i-1][j]
                    b[i][j] = 0
                else:
                    T[i][j] = T[i][j-1]
                    b[i][j] = -1
    return T[m][n], b


s, b = LCS_with_backtrace(string1, string2)

def print_LCS(b,string1,string2):
    m = len(string1)
    n = len(string2)
    a = ''
    while m > 1 or n > 1:
        if b[m][n] == 1:
            a = a + string1[m-1]
            m -= 1
            n -= 1
        elif b[m][n] == 0:
            m -= 1
        else:
            n -= 1
    return a[::-1]
    
print_LCS(b, string1, string2)

# Runtime O(m+n)